In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse import linalg

In [2]:
A = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]], 'float32')

In [14]:
evalues, evectors = linalg.eigsh(A, k=2, which="LM")

In [15]:
evalues

array([1.9999999, 2.9999998], dtype=float32)

In [16]:
evectors

array([[ 0.0000000e+00,  5.9604645e-08],
       [ 1.0000000e+00, -1.3120945e-07],
       [-2.0861626e-07, -1.0000000e+00]], dtype=float32)

In [17]:
np.matmul(A, evectors)

array([[ 0.0000000e+00,  5.9604645e-08],
       [ 2.0000000e+00, -2.6241889e-07],
       [-6.2584877e-07, -3.0000000e+00]], dtype=float32)

In [7]:
evectors[:,0] * 3

array([ 0.0000000e+00, -2.9999995e+00, -1.7881393e-07], dtype=float32)